## Power-law analysis in delta frequency band
Perform calculations of the Power Spectral Density (PSD) on log-log scale, where we evaluate the slope after have applied a low-pass filtering. We try several permutations to see if the evaluated slope of the PSD changes as a function of various parameters. As parameters we selects number of FFT lines: 256, 1024, 2048; order of the filter: 5 and 7 and fitting range: 0.x-8 Hz, and 0.x - 4 Hz with 4 Hz and 8 hz cut-off filter values. We perform this analysis in the three cortical areas analysed only in wakefulness.

In [5]:
import pandas as pd
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import os

C:\Users\37063\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
import warnings
warnings.filterwarnings("ignore")
warnings.resetwarnings()

In [2]:
!pip install --upgrade pandas

Requirement already up-to-date: pandas in c:\users\37063\anaconda3\lib\site-packages (2.0.3)


In [3]:
files = []

# Loop through files in the directory
for filename in os.listdir():
    if filename.endswith('W.csv'):
        
        files.append(filename)

C:\Users\37063\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
files

['Postcentral gyrus_W.csv',
 'Precentral gyrus_W.csv',
 'Superior temporal_gyrus_W.csv']

Perform low-pass filtering and calculate PSD on the signals

In [5]:
def norm(data):
  val = data/(data.sum())
  return val

def butter_lowpass(cutoff, order):
    nyq = 0.5 * 200
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, order):
    b, a = butter_lowpass(cutoff, order=order)
    y = signal.filtfilt(b, a, data)
    return y

def low_filtered(ar, cutoff, nperseg, order):

  def split_by_zero(arr):
    equals=np.isclose(arr, np.abs(arr).min())
    l=[]
    to_split=False
    for i in range(len(arr)):
        if equals[i] and i < len(arr)-1 and equals[i+1]:
            to_split=True
        elif not equals[i] and to_split:
            yield l
            l=[]
            to_split=False

        if not to_split:
            l.append(arr[i])
    yield l

  vector=[]
  s=[]
  for l in split_by_zero(ar):
    l=np.array(l)
    vector.append(l)
  f=[]
  for i in vector:
    x_filt = butter_lowpass_filter(i, cutoff, order)
    fft=signal.welch(x_filt, fs=200, window="hann", nperseg=nperseg, nfft=nperseg, noverlap=0)

    f.append(norm(fft[1]))
    ftm=np.mean(f,axis=0)
    fstd=np.std(f,axis=0)

  return ftm

In [6]:
from scipy.optimize import curve_fit

In [7]:
# Define a line for fitting the log-log PSD in 0.1-4 Hz frequency range
def line(a,x,b):
  return a*x+b

In [8]:
f = np.fft.rfftfreq(1024, 1/200)
f[1:20]

array([0.1953125, 0.390625 , 0.5859375, 0.78125  , 0.9765625, 1.171875 ,
       1.3671875, 1.5625   , 1.7578125, 1.953125 , 2.1484375, 2.34375  ,
       2.5390625, 2.734375 , 2.9296875, 3.125    , 3.3203125, 3.515625 ,
       3.7109375])

In [9]:
f = np.fft.rfftfreq(2048, 1/200)

In [22]:
start_end = [[(1, 5), (1, 20), (3, 41)], [(1, 11), (1, 41), (3, 81)]]

In [25]:
start_end[0][0][1]-start_end[0][0][0]

4

In [28]:
files

C:\Users\37063\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['Postcentral gyrus_W.csv',
 'Precentral gyrus_W.csv',
 'Superior temporal_gyrus_W.csv']

In [26]:
f[1:5]

array([0.09765625, 0.1953125 , 0.29296875, 0.390625  ])

In [21]:
f[1]-f[5]

0.09765625

In [15]:
cutoff = [4, 8]
orders = [5, 7]
fftlines = [256, 1024, 2048]
start_end = [[(1, 5), (1, 20), (3, 41)], [(1, 11), (1, 41), (3, 81)]]

params = []
fit = []

for file in files:
    
    df = pd.read_csv(file)
    
    # Iterate through fftlines, cutoff, order of the filter and fitting range

    for cut in cutoff:
                
        if cut == 4 or cut == 8: # for cutoff values equal to 4 or 8 let's set the fitting range 0.x - 4 Hz
        
            for fl, se in zip(fftlines, start_end[0]):
            
                freq = np.fft.rfftfreq(fl, 1/200)
            
                for order in orders:
                    params.append(np.array([cut, order, fl, freq[se[0]],freq[se[1]-1], se[1]-se[0]]))
                    FFT_df1 = df.apply(low_filtered, args=[cut, fl, order])
                    k = curve_fit(line, np.log(freq[se[0]:se[1]]), 
                              np.log(np.mean(FFT_df1[se[0]:se[1]], axis=1)),
                              sigma=np.std(FFT_df1[se[0]:se[1]], axis=1) / np.mean(FFT_df1[se[0]:se[1]], axis=1),
                              absolute_sigma=True)
        
                    fit.append(k)
            
            
            if cut == 8: # for cutoff value equal to 8, the fitting range we set 0.x - 8 Hz
            
                for fl, se in zip(fftlines, start_end[1]):

                    freq = np.fft.rfftfreq(fl, 1/200)
                
                    for order in orders:
                        params.append(np.array([cut, order, fl, freq[se[0]],freq[se[1]-1], se[1]-se[0]]))
                        FFT_df1 = df.apply(low_filtered, args=[cut, fl, order])
                        k = curve_fit(line, np.log(freq[se[0]:se[1]]), 
                                    np.log(np.mean(FFT_df1[se[0]:se[1]], axis=1)),
                                    sigma=np.std(FFT_df1[se[0]:se[1]], axis=1) / np.mean(FFT_df1[se[0]:se[1]], axis=1),
                                    absolute_sigma=True)
        
                        fit.append(k)
            
Post = fit[:18]
Prec = fit[18:36]
STG = fit[36:54]

C:\Users\37063\anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 1024 is greater than input length  = 800, using nperseg = 800
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\37063\anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 1024 is greater than input length  = 400, using nperseg = 400
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\37063\anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 1024 is greater than input length  = 800, using nperseg = 800
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\37063\anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 1024 is greater than input length  = 400, using nperseg = 400
  warnings.warn('nperseg = {0:d} is greater than input length '
C:\Users\37063\anaconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: n

In [29]:
params[0]

array([  4.     ,   5.     , 256.     ,   0.78125,   3.125  ,   4.     ])

In [ ]:
data = []
columns = ['Cut-off',"order", "FFT lines","Start","End","Num lines", "Beta", "Error", "Intercept", "Error1"]

# Loop through the values and append to the list
for (c, o, f, s, e, n), j in zip(params, fit):
        
    row_data = [c,o,f,s,e,n, j[0][0], np.sqrt(np.diag(j[1]))[0], j[0][1], np.sqrt(np.diag(j[1]))[1]]
    data.append(row_data)

# Create a DataFrame from the list
df = pd.DataFrame(data, columns=columns)

# Save obtained results as csv file
df.to_csv("Delta_band_analysis.csv")

# Print the resulting DataFrame
df

Statistical analysis of the parameters in three cortical area.
Paired t_test performed by pair comparisons, and the p-values are adjusted by multiplying all p-values by the lenght of the array of p-values (the number of tests performed in one cortical are)

In [3]:
from scipy.stats import ttest_ind_from_stats as ttest

In [65]:
stg_val = df.loc[36:54, ["Beta", "Error"]].reset_index(drop=True)
post_val = df.loc[:17, ["Beta", "Error"]].reset_index(drop=True)
prec_val = df.loc[18:35, ["Beta", "Error"]].reset_index(drop=True)

In [66]:
def statistics(data):
    p_values = []
    t_tests = []

    for i,j in zip(data["Beta"], data["Error"]):
    
        t_test, p_value = ttest(data["Beta"].mean(), data["Beta"].std(), 18,
                                i, j, 18)
    
        t_tests.append(t_test)
        p_values.append(p_value)
        
    return t_tests, p_values

C:\Users\37063\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
prec_m = [-0.24, -0.28, -0.26, -0.26, -0.34, -0.32, -0.49, -0.4, -0.36, -0.14, -0.03,
          -0.85, -0.42]
prec_std = [0.24, 0.13, 0.1, 0.36, 0.17, 0.15, 0.27, 0.16, 0.13, 0.49, 0.42, 0.27, 0.27]

#prec_m = [-0.24, -0.28, -0.26, -0.26, -0.34, -0.32, -0.49, -0.4, -0.36]
#prec_std = [0.24, 0.13, 0.1, 0.36, 0.17, 0.15, 0.27, 0.16, 0.13]

In [44]:
for i, j in zip(prec_m, prec_std):
    print(ttest(i, j, len(prec_m), -0.45, 0.15, len(prec_m)))

Ttest_indResult(statistic=2.675313695772239, pvalue=0.01323369351686219)
Ttest_indResult(statistic=3.0879658344813103, pvalue=0.005030628129169826)
Ttest_indResult(statistic=3.8, pvalue=0.0008720429519216258)
Ttest_indResult(statistic=1.756550621379892, pvalue=0.09174841007398928)
Ttest_indResult(statistic=1.749374540813571, pvalue=0.09300521183767914)
Ttest_indResult(statistic=2.209575122556873, pvalue=0.03692294241097679)
Ttest_indResult(statistic=-0.4669361306483164, pvalue=0.6447568626153846)
Ttest_indResult(statistic=0.8219949365267861, pvalue=0.4191715092895093)
Ttest_indResult(statistic=1.6348054417842237, pvalue=0.11513996639776386)
Ttest_indResult(statistic=2.1811523886639046, pvalue=0.03920152568871425)
Ttest_indResult(statistic=3.3954987505086622, pvalue=0.0023838182177491752)
Ttest_indResult(statistic=-4.669361306483165, pvalue=9.626126738760947e-05)
Ttest_indResult(statistic=0.35020209798623775, pvalue=0.7292418880977707)


In [47]:
post_m = [-0.43, -0.45, -0.43, -0.49, -0.51, -0.45, -0.74, -0.59, -0.53, -0.14, -0.13,
          -0.92, -0.47]
post_std = [0.18, 0.11, 0.09, 0.28, 0.17, 0.14, 0.21, 0.16, 0.13, 0.5, 0.43, 0.31, 0.3]

post_m = [-0.43, -0.45, -0.43, -0.49, -0.51, -0.45, -0.74, -0.59, -0.53]
post_std = [0.18, 0.11, 0.09, 0.28, 0.17, 0.14, 0.21, 0.16, 0.13]

In [48]:
for i, j in zip(post_m, post_std):
    print(ttest(i, j, len(post_m), -0.58, 0.14, len(post_m)))

Ttest_indResult(statistic=1.9733805434408151, pvalue=0.06598162873219694)
Ttest_indResult(statistic=2.190458229576748, pvalue=0.043646046207891906)
Ttest_indResult(statistic=2.703787956567493, pvalue=0.015650098702665437)
Ttest_indResult(statistic=0.8624833627499184, pvalue=0.40116457511148773)
Ttest_indResult(statistic=0.953560879291649, pvalue=0.3544911334936347)
Ttest_indResult(statistic=1.9697974618768102, pvalue=0.06642479487534092)
Ttest_indResult(statistic=-1.901829244200786, pvalue=0.07535460929230305)
Ttest_indResult(statistic=-0.1411081302575397, pvalue=0.8895458464403481)
Ttest_indResult(statistic=0.7851358838853195, pvalue=0.44384505010756303)


In [49]:
stg_m = [-0.65, -0.56, -0.56, -0.65, -0.54, -0.54, -0.78, -0.6, -0.6, 0.14, 0.26, -0.44, -0.31]
stg_std = [0.18, 0.1, 0.09, 0.26, 0.14, 0.13, 0.22, 0.14, 0.13, 0.38, 0.37, 0.31, 0.31]

stg_m = [-0.65, -0.56, -0.56, -0.65, -0.54, -0.54, -0.78, -0.6, -0.6]
stg_std = [0.18, 0.1, 0.09, 0.26, 0.14, 0.13, 0.22, 0.14, 0.13]

In [50]:
for i, j in zip(stg_m, stg_std):
    print(ttest(i, j, len(stg_m), -0.72, 0.14, len(stg_m)))

Ttest_indResult(statistic=0.92091092027238, pvalue=0.3707744404899538)
Ttest_indResult(statistic=2.789943329851661, pvalue=0.013110455139967354)
Ttest_indResult(statistic=2.884040487005325, pvalue=0.010792472574136349)
Ttest_indResult(statistic=0.7111497345525257, pvalue=0.48723035022785954)
Ttest_indResult(statistic=2.727411870290968, pvalue=0.014910050702365014)
Ttest_indResult(statistic=2.826489181987153, pvalue=0.012157931045157553)
Ttest_indResult(statistic=-0.6902684899626339, pvalue=0.49991660379202607)
Ttest_indResult(statistic=1.8182745801939793, pvalue=0.0877922105222801)
Ttest_indResult(statistic=1.884326121324769, pvalue=0.07782174159659198)


In [22]:
print(np.mean(stg_m), np.std(stg_m))

-0.6088888888888888 0.07233324799448797


#### some permutations

In [ ]:
colors = ["grey"]*3 + ["brown"]*3 + ["black"]*3
x_axis = [0, 1, 2]*3
f = [stg_val, post_val, prec_val]
fft_labels = [256, 1024, 2048]

fig, axes = plt.subplots(3, 1, figsize=(6, 10))

for ax, data in zip(axes, f):
    for index, color, pos in zip(range(0, len(data), 2), colors, x_axis):
        ax.errorbar(pos, np.abs(data.iloc[index]["Beta"]),
                    yerr=data.iloc[index]["Error"],
                    color=color, capsize=3, fmt="x", ms=10, ls="--")

    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(fft_labels, size=14)
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_yticklabels([0.0, 0.2, 0.4, 0.6, 0.8, 1.0], size=14)
    ax.set_ylabel("β", size=14)
    ax.set_ylim(0, 1)
    ax.grid()
plt.xlabel("Number of FFT lines", size=14)

plt.savefig("Low_freq_anal.jpg", dpi=200, bbox_inches="tight")

In [479]:
for index in zip(range(0, len(Post), 2)):
    print("Cut-off:{:.2f} Fit range: {:.2f}-{:.2f} (Hz)".format(p[index[0]][0],params[index[0]][3],params[index[0]][4]))

Cut-off:4.00 Fit range: 0.78-3.12 (Hz)
Cut-off:4.00 Fit range: 0.20-3.71 (Hz)
Cut-off:4.00 Fit range: 0.29-3.91 (Hz)
Cut-off:8.00 Fit range: 0.78-3.12 (Hz)
Cut-off:8.00 Fit range: 0.20-3.71 (Hz)
Cut-off:8.00 Fit range: 0.29-3.91 (Hz)
Cut-off:8.00 Fit range: 0.78-7.81 (Hz)
Cut-off:8.00 Fit range: 0.20-7.81 (Hz)
Cut-off:8.00 Fit range: 0.29-7.81 (Hz)
